# SFT Training Script

In [36]:
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
import os
import pandas as pd

In [37]:
# Empty VRAM cache
torch.cuda.empty_cache()

### Base Models

In [38]:
# model_type="microsoft/Phi-3-mini-4k-instruct"
# model_type="Qwen/Qwen2-0.5B"
# model_type="Qwen/Qwen2-1.5B"
model_type="VAGOsolutions/SauerkrautLM-1.5b"
output_model="function-calling-german-experiment"

### Load the German SFT Dataset

In [39]:
# dataset_df = pd.read_csv("function_calling_dataset.csv", sep=";")
dataset_df = pd.read_csv("german-dataset.csv", sep=";")
dataset_df

,function_name,command,parameters,prompt,completion,text
0,create_calendar_entry,Erstelle einen neuen Kalendareintrag für Vorle...,"(""Vorlesung Mittelhochdeutsch"", ""2023-03-09"", ...","Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Vorlesung Mittelhochdeutsch"", ""2023-03...","Unten steht ein Befehl des Benutzer, bitte wäh..."
1,create_calendar_entry,Plane Training im Fitnessstudio am 01.05.2023 ...,"(""Training im Fitnessstudio"", ""2023-05-01"", ""0...","Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Training im Fitnessstudio"", ""2023-05-0...","Unten steht ein Befehl des Benutzer, bitte wäh..."
2,create_calendar_entry,Trage Vorbesprechung Bachelorarbeit für den 14...,"(""Vorbesprechung Bachelorarbeit"", ""2023-07-14""...","Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Vorbesprechung Bachelorarbeit"", ""2023-...","Unten steht ein Befehl des Benutzer, bitte wäh..."
3,create_calendar_entry,Plane Training im Fitnessstudio am 04. April 2...,"(""Training im Fitnessstudio"", ""2024-04-04"", ""1...","Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Training im Fitnessstudio"", ""2024-04-0...","Unten steht ein Befehl des Benutzer, bitte wäh..."
4,create_calendar_entry,Trage Lineare Algebra Übung für den 09. Mai 20...,"(""Lineare Algebra Übung"", ""2023-05-09"", ""23:20...","Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_1>(""Lineare Algebra Übung"", ""2023-05-09"", ...","Unten steht ein Befehl des Benutzer, bitte wäh..."
...,...,...,...,...,...,...
2995,list_calendar_entries,Zeige alle Termine am 31.07.2023 an,"(""2023-07-31"", False)","Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2023-07-31"", False)<oc_end>\nFunktions...","Unten steht ein Befehl des Benutzer, bitte wäh..."
2996,list_calendar_entries,Zeige mir alle noch ausstehenden Termine für d...,"(""2026-06-29"", True)","Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2026-06-29"", True)<oc_end>\nFunktionsb...","Unten steht ein Befehl des Benutzer, bitte wäh..."
2997,list_calendar_entries,Liste alle noch offene Termine am 06.06.2025 auf,"(""2025-06-06"", True)","Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2025-06-06"", True)<oc_end>\nFunktionsb...","Unten steht ein Befehl des Benutzer, bitte wäh..."
2998,list_calendar_entries,Zeige alle Termine am 28. Dezember 2024 an,"(""2024-12-28"", False)","Unten steht ein Befehl des Benutzer, bitte wäh...","<oc_3>(""2024-12-28"", False)<oc_end>\nFunktions...","Unten steht ein Befehl des Benutzer, bitte wäh..."


In [43]:
dataset_df["text"] = dataset_df["prompt"] + dataset_df["completion"]
dataset_df["text"]

0       Unten steht ein Befehl des Benutzer, bitte wäh...
1       Unten steht ein Befehl des Benutzer, bitte wäh...
2       Unten steht ein Befehl des Benutzer, bitte wäh...
3       Unten steht ein Befehl des Benutzer, bitte wäh...
4       Unten steht ein Befehl des Benutzer, bitte wäh...
                              ...                        
2995    Below is the query from the users, please choo...
2996    Below is the query from the users, please choo...
2997    Below is the query from the users, please choo...
2998    Below is the query from the users, please choo...
2999    Below is the query from the users, please choo...
Name: text, Length: 6000, dtype: object

In [45]:
dataset = Dataset.from_pandas(dataset_df)
dataset[0]

{'function_name': 'create_calendar_entry',
 'command': 'Erstelle einen neuen Kalendareintrag für Vorlesung Mittelhochdeutsch am 09.03.2023 um 7 Uhr für 1,5 Stunden',
 'parameters': '("Vorlesung Mittelhochdeutsch", "2023-03-09", "07:00", 90)',
 'prompt': 'Unten steht ein Befehl des Benutzer, bitte wähle die korrekte Funktion aus und generiere Parameter, um die Funktion aufzurufen.\nBefehl: Erstelle einen neuen Kalendareintrag für Vorlesung Mittelhochdeutsch am 09.03.2023 um 7 Uhr für 1,5 Stunden\nAntwort: ',
 'completion': '<oc_1>("Vorlesung Mittelhochdeutsch", "2023-03-09", "07:00", 90)<oc_end>\nFunktionsbeschreibung: def create_calender_entry(title, date, time, duration):\n"""\nErstellt einen Kalendereintrag mit den angegebenen Details und ruft die Kalender-API auf.\n\nParameter:\ntitle (str): Der Titel des Kalendereintrags.\ndate (str): Das Datum des Kalendereintrags im Format \'YYYY-MM-DD\'.\ntime (str): Die Uhrzeit des Kalendereintrags im Format \'HH:MM\'.\nduration (int): Die Daue

### Load Base Model and Tokenizer

In [46]:
tokenizer = AutoTokenizer.from_pretrained(model_type)
model = AutoModelForCausalLM.from_pretrained(model_type, device_map="auto", trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [47]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear

### Add custom Special Tokens (Functional Tokens)

In [48]:
len(tokenizer)

151646

In [49]:
len(tokenizer.encode('<oc_1>("Team Sync", "2024-06-20", "10:00", 60)<oc_end>'))

34

In [50]:
new_tokens = ["<oc_1>", "<oc_2>", "<oc_3>", "<oc_end>"]
new_tokens = set(new_tokens) - set(tokenizer.vocab.keys())
new_tokens

{'<oc_1>', '<oc_2>', '<oc_3>', '<oc_end>'}

### Adapt Model Weights

In [51]:
tokenizer.add_tokens(list(new_tokens))
model.resize_token_embeddings(len(tokenizer))

Embedding(151650, 1536)

#### Add PAD Token

In [52]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [53]:
len(tokenizer)

151650

In [54]:
len(tokenizer.encode('<oc_1>("Team Sync", "2024-06-20", "10:00", 60)<oc_end>'))

29

### Train the Model

In [55]:
from trl import SFTTrainer

training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=4,
        optim = "adamw_8bit",
        learning_rate=5e-5,
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        save_strategy="epoch",
        logging_steps = 10,
        save_steps=1500,
        num_train_epochs=3,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        warmup_steps = 10,
        seed = 3407,
    )

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
)

trainer.train()

/home/ubuntu/fine-tuning/.venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/ubuntu/fine-tuning/.venv/lib/python3.10/site-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/home/ubuntu/fine-tuning/.venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/ubuntu/fine-tuning/.venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, th

Step,Training Loss
10,1.209000
20,0.403500
30,0.233600
40,0.198900
50,0.191700
60,0.190300
70,0.155500
80,0.189700
90,0.154700
100,0.189900


[codecarbon INFO @ 21:56:21] Energy consumed for RAM : 0.000049 kWh. RAM Power : 11.751500129699707 W
[codecarbon INFO @ 21:56:21] Energy consumed for all GPUs : 0.001009 kWh. Total GPU Power : 242.0692581750781 W
[codecarbon INFO @ 21:56:21] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:56:21] 0.001235 kWh of electricity used since the beginning.
[codecarbon INFO @ 21:56:36] Energy consumed for RAM : 0.000098 kWh. RAM Power : 11.751500129699707 W
[codecarbon INFO @ 21:56:36] Energy consumed for all GPUs : 0.002047 kWh. Total GPU Power : 249.1887408818675 W
[codecarbon INFO @ 21:56:36] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:56:36] 0.002499 kWh of electricity used since the beginning.
[codecarbon INFO @ 21:56:51] Energy consumed for RAM : 0.000147 kWh. RAM Power : 11.751500129699707 W
[codecarbon INFO @ 21:56:51] Energy consumed for all GPUs : 0.003084 kWh. Total GPU Power : 248.9815725441

TrainOutput(global_step=4500, training_loss=0.12159480693605211, metrics={'train_runtime': 1797.7969, 'train_samples_per_second': 10.012, 'train_steps_per_second': 2.503, 'total_flos': 3.842655598213939e+16, 'train_loss': 0.12159480693605211, 'epoch': 3.0})

### Save the Model and the Tokenizer

In [56]:
trainer.model.save_pretrained("multilingual-function-calling-model-1.5b", save_embedding_layers=True)

In [57]:
tokenizer.save_pretrained("multilingual-function-calling-model-1.5b")

('multilingual-function-calling-model-1.5b/tokenizer_config.json',
 'multilingual-function-calling-model-1.5b/special_tokens_map.json',
 'multilingual-function-calling-model-1.5b/vocab.json',
 'multilingual-function-calling-model-1.5b/merges.txt',
 'multilingual-function-calling-model-1.5b/added_tokens.json',
 'multilingual-function-calling-model-1.5b/tokenizer.json')

In [58]:
print(tokenizer.encode("<oc_1>"))
print(tokenizer.encode("<oc_2>"))
print(tokenizer.encode("<oc_3>"))
print(tokenizer.encode("<oc_irr>"))
print(tokenizer.encode("<oc_end>"))

[151647]
[151649]
[151648]
[27, 509, 51433, 81, 29]
[151646]


### Model Inference Example

In [69]:
from transformers import GenerationConfig

# prompt = """Below is the query from the users, please choose the correct function and generate the parameters to call the function.
# Query: Create an appointment Business Lunch on 21.06.2024 at 10:00 for 120 minutes.
# Response: """

prompt = """Unten befindet sich der Befehl des Benutzers, wähle bitte die passende Funktion aus und generiere die Parameter für die Funktion.
Befehl: {query}
Antwort: """.format(query="Erstelle einen neuen Termin mit Dieter zum Abendessen am 21.3.23 um 18 Uhr für 2 Stunden")

early_stopping_token = "<oc_end>"
eos_token_id = tokenizer.encode(early_stopping_token, add_special_tokens=False)[0]
eos_token_id

151646

In [70]:
generation_config = GenerationConfig(
    penalty_alpha=0.0,
    do_sample=False,
    top_k=1,
    temperature=0.0,
    repetition_penalty=2.0,
    max_new_tokens=64,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=eos_token_id  # Set the early stopping token ID
)

trained_model = trainer.model #.to('cuda', dtype=torch.float32)
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
outputs = trained_model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Unten befindet sich der Befehl des Benutzers, wähle bitte die passende Funktion aus und generiere die Parameter für die Funktion.
Befehl: Erstelle einen neuen Termin mit Dieter zum Abendessen am 21.3.23 um 18 Uhr für 2 Stunden
Antwort: <oc_1>("Erstgespräch Mittelhochdeutsch", "2026-09-21", '17:50", 12)<oc_end>


### (Optional) Generate DPO/ORPO Rejected Examples from the Fine-Tuned Model

In [61]:
# dpo_df = pd.read_csv("german-dataset-dpo-creation.csv", sep=";")
# dpo_df

In [62]:
# early_stopping_token = "<oc_end>"
# eos_token_id = tokenizer.encode(early_stopping_token, add_special_tokens=False)[0]
# trained_model = trainer.model.to('cuda', dtype=torch.float32)

# generation_config = GenerationConfig(
#     penalty_alpha=0.0,
#     do_sample=False,
#     top_k=1,
#     temperature=0.0,
#     repetition_penalty=2.0,
#     max_new_tokens=64,
#     pad_token_id=tokenizer.eos_token_id,
#     eos_token_id=eos_token_id  # Set the early stopping token ID
# )

# def create_rejected_parameters(prompt):
#     inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
#     outputs = trained_model.generate(**inputs, generation_config=generation_config)
#     output = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     print(output)
#     return output

In [63]:
# dpo_df["completion_parameters_rejected"] = dpo_df["prompt"].apply(create_rejected_parameters)

In [64]:
# dpo_df.to_csv("dpo-data.csv", sep=";", index=False)

In [65]:
# german_dpo_dataset = pd.read_csv("german-dpo-dataset.csv", sep=";", names=["prompt", "chosen", "rejected"], skiprows=1)

In [66]:
# german_dpo_dataset

In [67]:
# from datasets import Dataset
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from trl import DPOTrainer, DPOConfig

# dataset = Dataset.from_dict(german_dpo_dataset)
# dataset[0]

In [68]:
# len(tokenizer)